In [1]:
import tensorflow as tf
from tensorflow import keras

In [2]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
import numpy as np
import glob
import os 
import cv2

from keras.layers import Input,Conv3D,ConvLSTM2D,Conv3DTranspose
from keras.models import Sequential, Model
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.utils import img_to_array,load_img
from ModelSTAE import loadModel
# import imutils

In [4]:
store_image = []
video_source_path = './training_videos'
fps = 5

# list all the video names in the directory training_videos
train_videos = os.listdir(video_source_path)

# create a directory to store images after extracting
train_images_path = video_source_path +'/frames'

if not os.path.exists(train_images_path): 
    os.makedirs(train_images_path)

def store_in_array(image_path): 
    image = load_img(image_path) # keras function for loading images
    image = img_to_array(image) # img to 3-D numpy array
    
    # interpolation is the algorithm used for resizing an image
    image = cv2.resize(image, (227, 227), 
                        interpolation = cv2.INTER_AREA) 
    gray=0.2989*image[:,:,0]+0.5870*image[:,:,1]+0.1140*image[:,:,2]
    store_image.append(gray)

In [5]:
for video in train_videos:
    os.system( 'ffmpeg -i {}/{} -r 1/{}  {}/frames/%03d.jpg'.format(video_source_path,video,fps,video_source_path))
    images=os.listdir(train_images_path)
    for image in images:
        image_path=train_images_path + '/' + image
        store_in_array(image_path)

In [6]:
store_image=np.array(store_image)
a,b,c=store_image.shape
store_image.resize(b,c,a)
store_image=(store_image-store_image.mean())/(store_image.std())
store_image=np.clip(store_image,0,1)
np.save('training.npy',store_image)

In [7]:
training_data=np.load('training.npy')
frames=training_data.shape[2]
frames=frames-frames%10

training_data=training_data[:,:,:frames]
training_data=training_data.reshape(-1,227,227,10)
training_data=np.expand_dims(training_data,axis=4)
target_data=training_data.copy()

epochs=20
batch_size=1

In [8]:
model=loadModel()

callback_save = ModelCheckpoint("model.h5",
                                monitor="mean_squared_error", save_best_only=True)

callback_early_stopping = EarlyStopping(monitor='accuracy', patience=3)

print('Model has been loaded')

model.fit(training_data,target_data,
          batch_size=batch_size,
          epochs=epochs,
          callbacks = [callback_save,callback_early_stopping]
          )

Model has been loaded
Epoch 1/20
25/25 [==============================] - 19s 379ms/step - loss: 0.2346 - accuracy: 0.5274
Epoch 2/20
25/25 [==============================] - 10s 382ms/step - loss: 0.2029 - accuracy: 0.5443
Epoch 3/20
25/25 [==============================] - 9s 353ms/step - loss: 0.1999 - accuracy: 0.5447
Epoch 4/20
25/25 [==============================] - 10s 398ms/step - loss: 0.1992 - accuracy: 0.5450
Epoch 5/20
25/25 [==============================] - 9s 354ms/step - loss: 0.1497 - accuracy: 0.6208
Epoch 6/20
25/25 [==============================] - 10s 406ms/step - loss: 0.1013 - accuracy: 0.6969
Epoch 7/20
25/25 [==============================] - 10s 415ms/step - loss: 0.0909 - accuracy: 0.7120
Epoch 8/20
25/25 [==============================] - 9s 345ms/step - loss: 0.0872 - accuracy: 0.7141
Epoch 9/20
25/25 [==============================] - 9s 361ms/step - loss: 0.0840 - accuracy: 0.7164
Epoch 10/20
25/25 [==============================] - 9s 381ms/step - loss

In [9]:
model.save_weights('weights', save_format='tf')